In [7]:
from xarm.wrapper import XArmAPI
import numpy as np
from scipy.spatial.transform import Rotation as R

arm = XArmAPI('192.168.86.216')
arm.connect()
arm.motion_enable(enable=True)
arm.set_mode(0)
arm.set_state(0)

ROBOT_IP: 192.168.86.216, VERSION: v1.10.0, PROTOCOL: V1, DETAIL: 7,7,XS1303,AC1300,v1.10.0, TYPE1300: [1, 1]
change protocol identifier to 3


0

ControllerError, code: 23
ControllerError had clean


In [9]:
code, position = arm.get_position()
print(position)

[215.990036, 50.805801, 480.877808, -25.347137, -88.273659, -156.329758]


In [11]:
code, position = arm.get_position()
print(position)

[218.636948, 51.966072, 384.987274, -25.347137, -88.273717, -156.329758]


In [8]:
def create_transform(vec):
    x,y,z,rx,ry,rz = vec
    transform = np.eye(4)
    transform[:3,:3] = R.from_euler('xyz', [rx,ry,rz], degrees=True).as_matrix()
    transform[:3,3] = [x,y,z]
    return transform

In [9]:
def transform_to_vec(transform):
    x,y,z = transform[:3,3]
    rx,ry,rz = R.from_matrix(transform[:3,:3]).as_euler('xyz', degrees=True)
    return [x,y,z,rx,ry,rz]

In [14]:
wrist_to_iphone = create_transform([125,0,95,0,-15,0])

for i in range(1):
    
    code, current_pos = arm.get_position(is_radian=False)
    base_to_end_effector = create_transform(current_pos)
    
    action = [0,0,0,0,0,0]
    iphone_to_action = create_transform(action)
    
    full_transform = base_to_end_effector @ wrist_to_iphone @ iphone_to_action @ np.linalg.inv(wrist_to_iphone)
    
    new_pos = transform_to_vec(full_transform)
    
    arm.set_position(*new_pos, speed=100, mvacc=1000, wait=True)



In [8]:
arm.disconnect()